Constructing the company dataset, joining moneyball 1.1 successful and failed companies + scraping linkedin

In [6]:
import pandas as pd
import openpyxl

In [8]:
s_1 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/s-mball.xlsx", sheet_name = 'Company')
s_2 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/s-mball.xlsx", sheet_name = 'Long company descriptions')
s_3 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/s-mball.xlsx", sheet_name = 'Funding rounds')
s_4 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/s-mball.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_successful = []

In [9]:
df_s = pd.merge(s_1, s_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_s = pd.merge(df_s, s_3, on = ['org_uuid', 'org_name'])
df_s = pd.merge(df_s, s_4, on = ['org_uuid', 'org_name'])
df_s['founded_on'] = [c.strftime("%Y") for c in df_s['founded_on']]
df_s.dropna(inplace = True)
df_s = df_s[df_s['raised_amount_usd'] > 0]

In [10]:
df_inv_agg_s = df_s.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_s['investor_number_normalised']=(df_inv_agg_s['investor_count']-df_inv_agg_s['investor_count'].mean())/df_inv_agg_s['investor_count'].std()
df_inv_agg_s['raised_amount_usd_normalised']=(df_inv_agg_s['raised_amount_usd']-df_inv_agg_s['raised_amount_usd'].mean())/df_inv_agg_s['raised_amount_usd'].std()

In [11]:
investor_number = []
for c in df_inv_agg_s['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')

df_inv_agg_s['investor_number'] = investor_number
#df_inv_agg_s.drop(columns = ['investor_count','investor_number_normalised'], inplace = True)


In [12]:
raised_amount = []
for c in df_inv_agg_s['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')
df_inv_agg_s['investment_size'] = raised_amount


In [14]:
df_s = df_s.merge(df_inv_agg_s[['org_name', 'org_uuid','investor_number', 'investment_size']], how = 'left', on = ['org_name', 'org_uuid'])
df_s.drop(columns = 'category_list', inplace = True)
df_s.rename(columns = {'category_groups_list': 'category_list'}, inplace = True)
#df_s.to_excel(r'/Users/ipeksahbazoglu/VSCode/VelaPartners/successful_comp.xlsx', index=False)


In [19]:
fail_1 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/fail-mball.xlsx", sheet_name = 'Company')
fail_2 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/fail-mball.xlsx", sheet_name = 'Long company descriptions')
fail_3 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/fail-mball.xlsx", sheet_name = 'Funding rounds')
fail_4 = pd.read_excel("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/fail-mball.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_fail = []


In [20]:
df_f = pd.merge(fail_1, fail_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_f = pd.merge(df_f, fail_3, on = ['org_uuid', 'org_name'])
df_f = pd.merge(df_f, fail_4, on = ['org_uuid', 'org_name'])
df_f['founded_on'] = [c.strftime("%Y") for c in df_f['founded_on']]
df_f = df_f[df_f['raised_amount_usd'] > 0]
df_f.dropna(inplace = True)

In [22]:
df_inv_agg_f = df_f.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_f['investor_number_normalised']=(df_inv_agg_f['investor_count']-df_inv_agg_f['investor_count'].mean())/df_inv_agg_f['investor_count'].std()
df_inv_agg_f['raised_amount_usd_normalised']=(df_inv_agg_f['raised_amount_usd']-df_inv_agg_f['raised_amount_usd'].mean())/df_inv_agg_f['raised_amount_usd'].std()

In [23]:
investor_number = []
for c in df_inv_agg_f['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')


df_inv_agg_f['investor_number'] = investor_number
df_inv_agg_f.drop(columns = ['investor_count','investor_number_normalised'], inplace = True)
raised_amount = []
for c in df_inv_agg_f['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')


df_inv_agg_f['investment_size'] = raised_amount
df_inv_agg_f.drop(columns = ['raised_amount_usd_normalised', 'raised_amount_usd'], inplace = True)




In [24]:
df_f = df_f.merge(df_inv_agg_f, how = 'left', on = ['org_name', 'org_uuid'])
df_f.rename(columns = {'category_groups_list': 'category'}, inplace = True)
df_f = df_f[['org_uuid', 'org_name', 'founded_on',
       'category_list','country_code', 'city', 'long_description',
       'investment_type', 'investors_names', 'investors_uuids',
       'founder_linkedin_url', 'investor_number', 'investment_size']]
#df_f.to_excel(r'/Users/ipeksahbazoglu/VSCode/VelaPartners/failed_comp.xlsx', index=False)


In [47]:
inv = pd.read_csv('/Users/ipeksahbazoglu/VSCode/ProjectWeave/Datasets/investor_profiles.csv')
inv.dropna(inplace = True)
inv['investment_count_normalised']=(inv['investment_count']-inv['investment_count'].mean())/inv['investment_count'].std()

In [48]:
investment_count = []
for c in inv['investment_count_normalised']:
    if c <= 0.1:
        investment_count.append('D')
    elif c >= 0.1 and c < 1:
        investment_count.append('C')
    elif c >= 1 and c < 10:
        investment_count.append('B')
    elif c >= 10:
        investment_count.append('A')

inv['investment_count'] = investment_count
inv.drop(columns = 'investment_count_normalised', inplace = True)
#inv.to_excel(r'/Users/ipeksahbazoglu/VSCode/VelaPartners/investor-df.xlsx', index=False)
